# Modifications Detected in the HEK293T Data

This notebook creates a mass shift histogram figure.

## Setup

In [ ]:
import string
import warnings
import itertools
import collections
from pathlib import Path

import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyteomics.auxiliary
from spectrum_utils.spectrum import MsmsSpectrum
from ann_solo import reader, spectrum, utils

# plot styling
sns.set(context="paper", style="ticks")

# The search result files:
#solo_files = [f for f in snakemake.input if f.endswith("mztab")]
#fragger_files = [f for f in snakemake.input if f.endswith("tsv")]

solo_files = [f for f in Path("../../results").glob("*293T*.mztab")]
fragger_files = [f for f in Path("../../resources").glob("*293T*.tsv")]

# Group FDR parameters
max_fdr = 0.01
tol_mass = 0.1
tol_mode = "Da"
min_group_size = 20

## Helper functions to parse SSMs and PSMs

These functions were adapted from the original analysis:
https://github.com/bittremieux/ANN-SoLo/blob/master/notebooks/hek293_stats.ipynb

In [ ]:
def read_msfragger_psms(filename, **kwargs):
    """Read the MSFragger psms"""
    # The column names:
    cols = [
        "ScanID",
        "Precursor neutral mass (Da)",
        "Retention time (minutes)",
        "Precursor charge",
        "Hit rank",
        "Peptide Sequence",
        "Upstream Amino Acid",
        "Downstream Amino Acid",
        "Protein",
        "Matched fragment ions",
        "Total possible number of matched theoretical fragment ions",
        "Neutral mass of peptide (including any variable modifications) (Da)",
        "Mass difference",
        "Number of tryptic termini",
        "Number of missed cleavages",
        "Variable modifications detected",
        "Hyperscore",
        "Next score",
        "Intercept of expectation model",
        "Slope of expectation model",
    ]

    # Read in the data:
    psms = pd.read_csv(filename, sep="\t", header=None, names=cols)

    # Create columns to match the ANN-SoLo results:
    psms["charge"] = psms["Precursor charge"]
    psms["opt_ms_run[1]_cv_MS:1002217_decoy_peptide"] = psms["Protein"].str.contains("decoy_")
    psms["calc_mass_to_charge"] = 0
    psms["exp_mass_to_charge"] = psms["Mass difference"] / psms["charge"]
    psms["sequence"] = psms["Peptide Sequence"]
    psms["search_engine_score[1]"] = psms["Hyperscore"]

    # Create an identifier:
    psms["PSM_ID"] = psms.index

    # Keep only those columns:
    keep = [
        "sequence",
        "PSM_ID",
        "search_engine_score[1]",
        "charge",
        "exp_mass_to_charge",
        "calc_mass_to_charge",
        "opt_ms_run[1]_cv_MS:1002217_decoy_peptide",
    ]
    orig_len = len(psms)
    psms = psms.loc[:, keep].dropna()
    diff = orig_len - len(psms)
    if diff:
        msg = f"{diff} NAs rows were found!"
        warnings.warn(msg)
    
    return group_fdr(psms, **kwargs)


def group_fdr(psms, **kwargs):
    """Do the group FDR calculation"""
    matches = []
    for _, psm in psms.iterrows():
        spec = MsmsSpectrum(
            psm["PSM_ID"],
            psm["exp_mass_to_charge"],
            psm["charge"],
            np.array([0]),
            np.array([0]),
            peptide=psm["sequence"],
            is_decoy=psm["opt_ms_run[1]_cv_MS:1002217_decoy_peptide"],
        )

        match = spectrum.SpectrumSpectrumMatch(
            spec,
            spec,
            search_engine_score=psm["search_engine_score[1]"],
        )

        matches.append(match)

    passing = [s.query_identifier for s in utils.filter_group_fdr(matches, **kwargs)]
    psms = psms.set_index("PSM_ID").loc[passing, :].reset_index(drop=False)
    return psms


def read_matches(solo_files, fragger_files, **kwargs):
    """Read the SSMs or PSMs from the result files."""
    psms_list = []

    # Parse the ANN-SoLo results:
    for res_file in tqdm.tqdm(solo_files, desc="ANN-Solo", unit="files"):
        psms_df = reader.read_mztab_ssms(res_file)
        psms_df["search_engine"] = "ANN-SoLo"
        psms_list.append(psms_df)

    # Parse the MSFragger results:
    for res_file in tqdm.tqdm(fragger_files, desc="MSFragger", unit="files"):
        psms_df = read_msfragger_psms(res_file, **kwargs)
        psms_df["search_engine"] = "MSFragger"
        psms_list.append(psms_df)

    psms = pd.concat(psms_list).reset_index(drop=True)
    psms["mass_diff"] = (
        psms["exp_mass_to_charge"] - psms["calc_mass_to_charge"]
    ) * psms["charge"]

    keep = [
        "search_engine",
        "sequence",
        "charge",
        "exp_mass_to_charge",
        "calc_mass_to_charge",
        "search_engine_score[1]",
        "mass_diff",
    ]
    return psms.loc[:, keep]


def get_mass_groups(psms, tol_mass, tol_mode):
    """Calculate mass groups"""
    # start with the highest ranked PSM
    mass_groups = []
    psms_remaining = psms.sort_values("search_engine_score[1]", ascending=False)
    while len(psms_remaining) > 0:
        # find all remaining PSMs within the precursor mass window
        mass_diff = psms_remaining["mass_diff"].iloc[0]
        if tol_mass is None or tol_mode not in ("Da", "ppm"):
            psms_selected = psms_remaining
        elif tol_mode == "Da":
            psms_selected = psms_remaining[
                (psms_remaining["mass_diff"] - mass_diff).abs() <= tol_mass
            ]
        elif tol_mode == "ppm":
            psms_selected = psms_remaining[
                (psms_remaining["mass_diff"] - mass_diff).abs()
                / psms_remaining["exp_mass_to_charge"]
                * 10 ** 6
                <= tol_mass
            ]
        mass_groups.append(psms_selected)
        # exclude the selected PSMs from further selections
        psms_remaining.drop(psms_selected.index, inplace=True)

    mass_group_stats = []
    for mass_group in mass_groups:
        mass_group_stats.append(
            (
                mass_group["mass_diff"].median(),
                mass_group["mass_diff"].mean(),
                len(mass_group),
            )
        )
    mass_group_stats = pd.DataFrame.from_records(
        mass_group_stats, columns=["mass_diff_median", "mass_diff_mean", "num_psms"]
    )
    return mass_group_stats

## Run the Analysis

In [ ]:
psms = read_matches(
    solo_files,
    fragger_files,
    fdr=max_fdr,
    tol_mass=tol_mass,
    tol_mode=tol_mode,
    min_group_size=min_group_size,
)

mass_groups = {}
for search_engine, df in psms.groupby("search_engine"):
    mass_groups[search_engine] = get_mass_groups(df, tol_mass, tol_mode)

## Create the plot

In [ ]:
fig, axes = plt.subplots(2, 1, sharey=True, figsize=(7, 3))
for ax, (engine, df), lab in zip(axes, mass_groups.items(), ["SSMs", "PSMs"]):
    mod = df.loc[df["mass_diff_median"].abs() > tol_mass, :]
    ax.vlines(mod["mass_diff_median"], 0, mod["num_psms"], linewidth=1)
    ax.set_xlim(-50, 200)
    ax.set_ylim(0)
    ax.set_ylabel(engine + " " + lab)
    sns.despine(ax=ax)

axes[-1].set_xlabel("Mass Shift (Da)")
plt.tight_layout()

result_dir = Path("../results")
result_dir.mkdir(exist_ok=True)

plt.savefig(result_dir / "mass_shifts.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()